In [4]:
import os
import warnings
import sys

warnings.filterwarnings('ignore')
current_dir = %pwd

parent_dir = os.path.abspath(os.path.join(current_dir, '../..'))
sys.path.append(parent_dir)

In [5]:
from src.main.models.transformers import Transformer
from config import config
from src.main.pipeline.functions import clean_text, stop_words_removal
from src.main.utilities.utils import get_dataset, split_train_val_test
import numpy as np

inputs, targets = get_dataset(one_hot=False)

checkpoint = "uclanlp/newsbert"

training_args = {
    'output_dir': config.RESULTS_DIRECTORY.format(checkpoint),
    'num_train_epochs': 5,
    'per_device_train_batch_size': 16,
    'per_device_eval_batch_size': 16,
    'learning_rate': 2e-5,
    'warmup_steps': 500,
    'weight_decay': 0.01,
    'logging_dir': config.LOGS_PATH.format(checkpoint),
    'logging_steps': 500,
    'evaluation_strategy': 'epoch',
    'save_strategy': 'epoch',
    'load_best_model_at_end': True,
    'metric_for_best_model': 'accuracy'
}

newsBert = Transformer(checkpoint=checkpoint, **training_args)
newsBert.pipeline = [clean_text, stop_words_removal]
inputs = newsBert.run_pipeline(inputs, save=True)
targets = np.vectorize(config.label2id.get)(targets)

x_train, x_val, x_test, y_train, y_val, y_test = split_train_val_test(inputs, targets, validation_size=0.2, test_size=0.1)

newsBert.prepare_dataset(x_train, x_val, y_train, y_val)
newsBert.fit(x_train, y_train)

test_dataset = newsBert.prepare_test_dataset(x_test, y_test)
newsBert.save_results(test_dataset)
newsBert.save_model()